# Packages
You can use the `#r` statement to import packages from nuget. It is possible to omit the package version but specifying a version prevents future breakage by package updates. If you change versions or if the package is updated when you haven't specified a version, you will need to restart the kernel.

In [10]:
#r "nuget: Plotly.NET, 3.0.1"
#r "nuget: Plotly.NET.CSharp, 0.8.0"
#r "nuget: Plotly.NET.Interactive, 3.0.1"
#r "nuget: CsvHelper, 30.0.1"

Installed Packages CsvHelper, 30.0.1 Plotly.NET, 3.0.1 Plotly.NET.CSharp, 0.8.0 Plotly.NET.Interactive, 3.0.1

# Imports
You can specify using statements just as you would in C#. A few common namespaces like `System` and `System.Text` are imported by default.

In [13]:
using System.IO;
using System.Globalization;
using Plotly.NET.CSharp;
using CsvHelper;

# Data
Now you can use your favourite .NET tooling to get some data. In this example, we'll use `CsvHelper` to read a CSV file containing all FIFA world cup matches until 2018.

In [7]:
public class Match {
    
    public DateTime Date;
    public Side Home;
    public Side Away;
    public int Goals => (Home?.Score + Away?.Score) ?? 0;
}

public class Side {
    public string Name;
    public int Score;
}

private IEnumerable<Match> LoadData() {
    using(var reader = new StreamReader("../data/matches_1930_2018.csv")) 
    using(var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
    {
        csv.Read();
        csv.ReadHeader();
        while(csv.Read()) {
            yield return new Match() {
                Date = csv.GetField<DateTime>("Date"),
                Home = ReadSide(csv, "home_"),
                Away = ReadSide(csv, "away_")
            };
        }
    }
}

private Side ReadSide(CsvReader csv, string prefix) {
    return new Side() {
        Name = csv.GetField(prefix + "team"),
        Score = csv.GetField<int>(prefix + "score"),
    };
}

var data = LoadData().ToList();
Console.WriteLine($"Loaded {data.Count} matches");


Loaded 900 matches


# Chart
Let's wrap this up by creating a chart. Do you feel football has been getting a bit dull lately? Let's check the data to plot the average number of goals per match:

In [19]:
// Use some linq to extract the information we need
var grouped =
    data
    .GroupBy(m => m.Date.Year)
    .Select(g => new {
        Year = g.Key,
        AverageGoals = g.Average(m => (double)m.Goals)
    })
    .OrderBy(g => g.Year)
    .ToList();


var xData = grouped.Select(g => g.Year).ToList();
var yData = grouped.Select(g => g.AverageGoals).ToList();

var chart = 
    Plotly.NET.CSharp.Chart.Point<int, double, string>(xData, yData, Name: "")
    .WithTitle("Average goals per match over time");

display(chart);


<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

# Conclusion
It turns out we were right... if we have been watching football since the fifties that is. But more importantly, we used C# to do it!